In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train.shape

(42000, 785)

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
scaler = MinMaxScaler()

In [7]:
pixel_data = train.drop('label',axis=1)
pixel_data = scaler.fit_transform(pixel_data)

In [8]:
labels = train['label']

In [9]:
labels.reshape((42000,1))

C:\Users\karab\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


array([[1],
       [0],
       [1],
       ..., 
       [7],
       [6],
       [9]], dtype=int64)

In [10]:
#Need to change this to a hot ecnoded label
labels_he = np.zeros((labels.size, labels.max()+1))
labels_he[np.arange(labels.size),labels] = 1
labels_he.shape

(42000, 10)

In [11]:
#Placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [12]:
#Variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [13]:
#Create Graph Operations
y = tf.matmul(x,W) + b

In [14]:
#Loss Function
y_true = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y))

In [15]:
#Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
train = optimizer.minimize(cross_entropy)

In [16]:
#Create Session
init = tf.global_variables_initializer()

In [17]:
#def get_next_batch(batch_size=64,x_data,labels=None,batch_num=0):
#    batch_x = []

In [18]:
saver = tf.train.Saver()

In [19]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for step in range(500):
        #TO DO - get this to run in batches, not as a whole training set
        #make into batches
        #current_batch = 1
        #batch_x, batch_y = get_next_batch(100,pixel_data,labels_he)
        if step%100==0:
            print(step)
            
        sess.run(train,feed_dict={x:pixel_data,y_true:labels_he})
        
    #Evaluate the model
    correct_prediction = tf.equal(tf.argmax(y,1), tf.arg_max(y_true,1))
    
    #[True, False, True] --> [1,0,1,...]
    acc = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    print(sess.run(acc,feed_dict={x:pixel_data,y_true:labels_he}))
    
    saver.save(sess,'./MINSTmodel.ckpt')

0
100
200
300
400
Instructions for updating:
Use `argmax` instead
0.91719


In [20]:
#lets use the model now to predict
test = pd.read_csv('test.csv')

In [21]:
test_normalized = scaler.fit_transform(test)

In [22]:
predictions = tf.nn.softmax(tf.matmul(x,W) + b)

In [35]:
with tf.Session() as sess:
    
    saver.restore(sess,'MINSTmodel.ckpt')

    prediction=tf.argmax(y,1)
    pred = prediction.eval(feed_dict={x: test_normalized})

INFO:tensorflow:Restoring parameters from MINSTmodel.ckpt


In [38]:
test = pd.DataFrame()

In [39]:
test['Label'] = pred

In [46]:
test['Label'].size

28000

In [50]:
test['ImageId'] = np.arange(1,test['Label'].size+1)

In [51]:
test.head()

,Label,ImageId
0,2,1
1,0,2
2,9,3
3,9,4
4,3,5


In [52]:
test.to_csv('eval.csv')